# Comprehensive Report on Model Development and Improvement for Sentiment Analysis

## Introduction

The goal of this project was to develop a robust sentiment analysis model capable of classifying comments into three categories: Neutral, Positive, and Negative.

Initially, the dataset exhibited significant class imbalance, with a majority of comments labeled as Positive and Neutral, while the Negative class was underrepresented. This posed challenges in achieving balanced performance across all classes. Furthermore, overlapping data splits and inadequate preprocessing led to overfitting and unreliable test results.

This report details the step-by-step process undertaken to address these issues and achieve high accuracy and balanced performance.

---

## Initial Challenges

At the beginning of the project, the results were unrealistic due to overlapping datasets and class imbalance:

1. **Overlapping Data Splits**:
   - The original splits for training, validation, and testing were not completely disjoint. This overlap allowed the model to "memorize" examples rather than generalize.
   - As a result, validation and test accuracy were artificially inflated, indicating a lack of robustness.

2. **Class Imbalance**:
   - The Neutral class was underrepresented, leading to poor precision and recall for this category.
   - The model heavily favored the Positive class, which dominated the dataset.

3. **Model Complexity and Overfitting**:
   - Early models, such as simple LSTMs and CNNs, performed well on training and validation but failed to generalize to unseen test data.

---

## Steps Taken to Address the Issues

To resolve these challenges and develop a reliable model, the following steps were undertaken:

### 1. Non-Overlapping Dataset Splits
   - The dataset was re-split into training, validation, and test sets, ensuring no overlap among them.
   - The new splits were analyzed to confirm balanced class distributions within each subset.

### 2. Class Weighting and Balancing
   - Class weights were computed using the `compute_class_weight` function to address the imbalance.
   - This ensured that the model assigned appropriate importance to the underrepresented Neutral and Negative classes during training.

### 3. Experimentation with Model Architectures
   - Various architectures were tested, including:
     - LSTM-based models.
     - CNNs.
     - Hybrid CNN-LSTM models.
     - Attention mechanisms to focus on critical parts of the input sequence.
   - Each model was evaluated through cross-validation to ensure consistent performance across multiple folds.

### 4. Use of Pre-Trained Embeddings
   - GloVe embeddings (300-dimensional) were incorporated into the final model to provide semantic richness and improve generalization.
   - Pre-trained embeddings enabled the model to leverage external knowledge and better understand the context of words.

### 5. Regularization Techniques
   - Dropout layers and early stopping were applied to prevent overfitting.
   - Hyperparameters such as LSTM units, learning rates, and dropout rates were fine-tuned for optimal performance.

---

## Final Model and Results

The final model combined a Bidirectional LSTM with an Attention mechanism and GloVe embeddings. This architecture effectively captured contextual dependencies and focused on critical words in the input.

### Results:
1. **Test Accuracy**: 95.3%
2. **Class Performance**:
   - Neutral: Precision (90%), Recall (93%), F1-score (91%).
   - Positive: Precision (97%), Recall (96%), F1-score (97%).
   - Negative: Precision (92%), Recall (94%), F1-score (93%).
3. **Macro Average F1-Score**: 94%
4. **Weighted Average F1-Score**: 95%

---

## Key Improvements

1. **Neutral Class Performance**:
   - The Neutral class performance improved significantly due to class weighting and the inclusion of attention mechanisms.

2. **Reduced Overfitting**:
   - The test accuracy closely matched validation accuracy, indicating reduced overfitting and better generalization.

3. **Enhanced Semantic Understanding**:
   - Pre-trained embeddings enhanced semantic understanding, contributing to balanced performance across all classes.

---

## Conclusion

The project successfully developed a robust sentiment analysis model with high accuracy and balanced performance across all classes.

The systematic approach—addressing data leakage, class imbalance, and overfitting—proved effective in resolving early challenges. The final model can serve as a foundation for further applications or as a benchmark for exploring advanced techniques like transformer-based architectures.

### Future Work
- Hyperparameter tuning.
- Exploring ensemble models.
- Testing on external datasets to further validate generalization.
- Extending the model to handle multilingual data.


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = "/content/us_election_comments_relabelled_final (1).csv"  # Update with your file's path
data = pd.read_csv(file_path)

# Filter relevant comments and extract necessary columns
filtered_data = data[data['is_relevant'] == True][['comment', 'label']]
comments = filtered_data['comment'].values
labels = filtered_data['label'].values

# Convert labels to categorical (for classification)
labels = labels + 1  # Shift labels from (-1, 0, 1) to (0, 1, 2)
labels = to_categorical(labels, num_classes=3)

# Parameters
VOCAB_SIZE = 30000  # Increased vocabulary size
EMBEDDING_DIM = 300  # Larger embeddings for better feature representation
MAX_LENGTH = 200  # Allow longer sequences
BATCH_SIZE = 64  # Larger batch size for stability
EPOCHS = 30  # More epochs for better training

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(comments, labels, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the sequences
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

# Build the improved LSTM model
model = Sequential([
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
    Bidirectional(LSTM(256, activation='tanh', return_sequences=True)),  # First LSTM layer
    Dropout(0.5),
    Bidirectional(LSTM(128, activation='tanh')),  # Second LSTM layer
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Output layer with softmax for classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Suitable for multi-class classification
    metrics=['accuracy']  # Use accuracy as the primary metric
)

# Summary of the model
model.summary()

# Add early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the model in the recommended format
model.save("lstm_sentiment_model.keras")

# Test predictions
sample_comments = [
    "I am hopeful about this election!",
    "This is the worst election in history.",
    "I don't have a strong opinion about the election."
]
sample_sequences = tokenizer.texts_to_sequences(sample_comments)
sample_padded = pad_sequences(sample_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')

predictions = model.predict(sample_padded)
print("Predictions (probabilities):", predictions)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1) - 1  # Shift back to original labels (-1, 0, 1)
print("Predicted Labels:", predicted_labels)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 20s 95ms/step - accuracy: 0.6287 - loss: 0.8571 - val_accuracy: 0.8553 - val_loss: 0.4219
Epoch 2/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 9s 91ms/step - accuracy: 0.8957 - loss: 0.3062 - val_accuracy: 0.9064 - val_loss: 0.2802
Epoch 3/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 10s 88ms/step - accuracy: 0.9404 - loss: 0.1727 - val_accuracy: 0.9637 - val_loss: 0.1874
Epoch 4/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 9s 88ms/step - accuracy: 0.9488 - loss: 0.2320 - val_accuracy: 0.9631 - val_loss: 0.1484
Epoch 5/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 9s 88ms/step - accuracy: 0.9793 - loss: 0.0879 - val_accuracy: 0.9797 - val_loss: 0.0886
Epoch 6/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 9s 92ms/step - accuracy: 0.9913 - loss: 0.0488 - val_accuracy: 0.9871 - val_loss: 0.0595
Epoch 7/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.9927 - loss: 0.0344 - val_accuracy: 0.9865 - val_loss: 0.0776
Epoch 8/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 10s 86ms/step - accuracy: 0.9958 - loss: 0.0258 - val_

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = "/content/us_election_comments_relabelled_final (1).csv"  # Update with your file's path
data = pd.read_csv(file_path)

# Filter relevant comments and extract necessary columns
filtered_data = data[data['is_relevant'] == True][['comment', 'label']]
comments = filtered_data['comment'].values
labels = filtered_data['label'].values

# Convert labels to categorical (for classification)
labels = labels + 1  # Shift labels from (-1, 0, 1) to (0, 1, 2)
labels = to_categorical(labels, num_classes=3)

# Parameters
VOCAB_SIZE = 30000  # Increased vocabulary size
EMBEDDING_DIM = 300  # Larger embeddings for better feature representation
MAX_LENGTH = 200  # Allow longer sequences
BATCH_SIZE = 64  # Larger batch size for stability
EPOCHS = 30  # More epochs for better training

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(comments, labels, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the sequences
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

# Build the improved LSTM model
model = Sequential([
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
    Bidirectional(LSTM(256, activation='tanh', return_sequences=True)),  # First LSTM layer
    Dropout(0.5),
    Bidirectional(LSTM(128, activation='tanh')),  # Second LSTM layer
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Output layer with softmax for classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Suitable for multi-class classification
    metrics=['accuracy']  # Use accuracy as the primary metric
)

# Summary of the model
model.summary()

# Add early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the model in the recommended format
model.save("lstm_sentiment_model.keras")

# Test predictions
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels
y_true = np.argmax(y_test, axis=1)  # Convert one-hot encoding to class labels

# Calculate additional metrics
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Negative', 'Neutral', 'Positive']))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# Test with sample comments
sample_comments = [
    "I am hopeful about this election!",
    "This is the worst election in history.",
    "I don't have a strong opinion about the election."
]
sample_sequences = tokenizer.texts_to_sequences(sample_comments)
sample_padded = pad_sequences(sample_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')

predictions = model.predict(sample_padded)
print("Predictions (probabilities):", predictions)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1) - 1  # Shift back to original labels (-1, 0, 1)
print("Predicted Labels:", predicted_labels)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 15s 94ms/step - accuracy: 0.6530 - loss: 0.8356 - val_accuracy: 0.8793 - val_loss: 0.3487
Epoch 2/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 10s 89ms/step - accuracy: 0.8980 - loss: 0.3320 - val_accuracy: 0.9150 - val_loss: 0.2705
Epoch 3/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 9s 90ms/step - accuracy: 0.9139 - loss: 0.2937 - val_accuracy: 0.9415 - val_loss: 0.1591
Epoch 4/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 9s 88ms/step - accuracy: 0.9449 - loss: 0.1381 - val_accuracy: 0.9600 - val_loss: 0.1176
Epoch 5/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 10s 87ms/step - accuracy: 0.9874 - loss: 0.0537 - val_accuracy: 0.9772 - val_loss: 0.0915
Epoch 6/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 11s 92ms/step - accuracy: 0.9910 - loss: 0.0412 - val_accuracy: 0.9809 - val_loss: 0.0815
Epoch 7/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.9970 - loss: 0.0168 - val_accuracy: 0.9809 - val_loss: 0.1009
Epoch 8/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.9921 - loss: 0.0352 - val_

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = "/content/us_election_comments_relabelled_final (1).csv"  # Update with your file's path
data = pd.read_csv(file_path)

# Filter relevant comments and extract necessary columns
filtered_data = data[data['is_relevant'] == True][['comment', 'label']]
comments = filtered_data['comment'].values
labels = filtered_data['label'].values

# Convert labels to categorical (for classification)
labels = labels + 1  # Shift labels from (-1, 0, 1) to (0, 1, 2)
labels = to_categorical(labels, num_classes=3)

# Parameters
VOCAB_SIZE = 30000  # Increased vocabulary size
EMBEDDING_DIM = 300  # Larger embeddings for better feature representation
MAX_LENGTH = 200  # Allow longer sequences
BATCH_SIZE = 64  # Larger batch size for stability
EPOCHS = 30  # More epochs for better training

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(comments, labels, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the sequences
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

# Build the CNN model
model = Sequential([
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
    Conv1D(128, 5, activation='relu'),  # Convolutional layer with 128 filters and kernel size 5
    GlobalMaxPooling1D(),  # Global max pooling to reduce dimensionality
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Output layer with softmax for classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Suitable for multi-class classification
    metrics=['accuracy']  # Use accuracy as the primary metric
)

# Summary of the model
model.summary()

# Add early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the model in the recommended format
model.save("cnn_sentiment_model.keras")

# Test predictions
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels
y_true = np.argmax(y_test, axis=1)  # Convert one-hot encoding to class labels

# Calculate additional metrics
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Negative', 'Neutral', 'Positive']))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# Test with sample comments
sample_comments = [
    "I am hopeful about this election!",
    "This is the worst election in history.",
    "I don't have a strong opinion about the election."
]
sample_sequences = tokenizer.texts_to_sequences(sample_comments)
sample_padded = pad_sequences(sample_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')

predictions = model.predict(sample_padded)
print("Predictions (probabilities):", predictions)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1) - 1  # Shift back to original labels (-1, 0, 1)
print("Predicted Labels:", predicted_labels)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ ?                           │     0 (unbuilt) │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 11s 56ms/step - accuracy: 0.5875 - loss: 0.9272 - val_accuracy: 0.8467 - val_loss: 0.5178
Epoch 2/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8637 - loss: 0.4027 - val_accuracy: 0.9723 - val_loss: 0.1131
Epoch 3/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9713 - loss: 0.0929 - val_accuracy: 0.9877 - val_loss: 0.0484
Epoch 4/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9932 - loss: 0.0306 - val_accuracy: 0.9951 - val_loss: 0.0337
Epoch 5/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9972 - loss: 0.0148 - val_accuracy: 0.9951 - val_loss: 0.0467
Epoch 6/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9994 - loss: 0.0055 - val_accuracy: 0.9951 - val_loss: 0.0493
Epoch 7/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9997 - loss: 0.0034 - val_accuracy: 0.9951 - val_loss: 0.0513
Epoch 8/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9999 - loss: 0.0022 - val_accurac

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, LSTM, GlobalMaxPooling1D, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = "/content/us_election_comments_relabelled_final (1).csv"  # Update with your file's path
data = pd.read_csv(file_path)

# Filter relevant comments and extract necessary columns
filtered_data = data[data['is_relevant'] == True][['comment', 'label']]
comments = filtered_data['comment'].values
labels = filtered_data['label'].values

# Convert labels to categorical (for classification)
labels = labels + 1  # Shift labels from (-1, 0, 1) to (0, 1, 2)
labels = to_categorical(labels, num_classes=3)

# Parameters
VOCAB_SIZE = 30000  # Increased vocabulary size
EMBEDDING_DIM = 300  # Larger embeddings for better feature representation
MAX_LENGTH = 200  # Allow longer sequences
BATCH_SIZE = 64  # Larger batch size for stability
EPOCHS = 30  # More epochs for better training

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(comments, labels, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the sequences
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

# Build the corrected Hybrid CNN + LSTM model
model = Sequential([
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
    Conv1D(128, 5, activation='relu'),  # CNN layer to extract n-gram features
    Bidirectional(LSTM(128, activation='tanh', return_sequences=True)),  # LSTM layer for sequential context
    GlobalMaxPooling1D(),  # Global max pooling after LSTM
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Output layer with softmax for classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Suitable for multi-class classification
    metrics=['accuracy']  # Use accuracy as the primary metric
)

# Summary of the model
model.summary()

# Add early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the model in the recommended format
model.save("hybrid_cnn_lstm_sentiment_model.keras")

# Test predictions
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels
y_true = np.argmax(y_test, axis=1)  # Convert one-hot encoding to class labels

# Calculate additional metrics
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Negative', 'Neutral', 'Positive']))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# Test with sample comments
sample_comments = [
    "I am hopeful about this election!",
    "This is the worst election in history.",
    "I don't have a strong opinion about the election."
]
sample_sequences = tokenizer.texts_to_sequences(sample_comments)
sample_padded = pad_sequences(sample_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')

predictions = model.predict(sample_padded)
print("Predictions (probabilities):", predictions)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1) - 1  # Shift back to original labels (-1, 0, 1)
print("Predicted Labels:", predicted_labels)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_6 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_2               │ ?                           │     0 (unbuilt) │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.6135 - loss: 0.8954 - val_accuracy: 0.8855 - val_loss: 0.3274
Epoch 2/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.8987 - loss: 0.2770 - val_accuracy: 0.9495 - val_loss: 0.1390
Epoch 3/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.9834 - loss: 0.0672 - val_accuracy: 0.9846 - val_loss: 0.0660
Epoch 4/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.9962 - loss: 0.0173 - val_accuracy: 0.9889 - val_loss: 0.0539
Epoch 5/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.9984 - loss: 0.0071 - val_accuracy: 0.9865 - val_loss: 0.0809
Epoch 6/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.9997 - loss: 0.0026 - val_accuracy: 0.9920 - val_loss: 0.0653
Epoch 7/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.9993 - loss: 0.0030 - val_accuracy: 0.9914 - val_loss: 0.0687
Epoch 8/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.9995 - loss: 8.9350e-04 - val_

In [ ]:
import pandas as pd

# Load the dataset
file_path = "/content/us_election_comments_relabelled_final (1).csv"  # Update with your file's path
data = pd.read_csv(file_path)

# Display basic info about the dataset
print("Basic Info About the Dataset:")
data.info()

# Display the first few rows of the dataset
print("\nFirst Few Rows of the Dataset:")
print(data.head())

# Count the number of rows
num_rows = len(data)
print(f"\nNumber of Rows in the Dataset: {num_rows}")

# Check for missing values
missing_values = data.isnull().sum()
print("\nMissing Values in Each Column:")
print(missing_values)

# Class distribution for sentiment labels
if 'label' in data.columns:
    class_distribution = data['label'].value_counts()
    print("\nClass Distribution for Sentiment Labels:")
    print(class_distribution)
else:
    print("\nNo 'label' column found in the dataset.")

# Percentage distribution of classes
if 'label' in data.columns:
    class_percentage = data['label'].value_counts(normalize=True) * 100
    print("\nPercentage Distribution for Sentiment Labels:")
    print(class_percentage)

# Identify unique subreddits if relevant
if 'subreddit' in data.columns:
    unique_subreddits = data['subreddit'].nunique()
    print(f"\nNumber of Unique Subreddits: {unique_subreddits}")


Basic Info About the Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8700 entries, 0 to 8699
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           8700 non-null   object
 1   comment      8700 non-null   object
 2   created_utc  8700 non-null   object
 3   score        8700 non-null   int64 
 4   subreddit    8700 non-null   object
 5   is_relevant  8700 non-null   bool  
 6   label        8700 non-null   int64 
dtypes: bool(1), int64(2), object(4)
memory usage: 416.4+ KB

First Few Rows of the Dataset:
        id                                            comment  \
0  lvfp7bg  to save you a click, ralston says it will be r...   
1  lvezm7f  Here's a preview of the story: \n\nSeven battl...   
2  lvf4uks  Well how absurd is it that the entire country'...   
3  lvel24i  \nAs a reminder, this subreddit [is for civil ...   
4  lveohui  >“At least 7 of the keys, maybe 8, clearly fav...   

           

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# Reload the trained model (to ensure it's in a valid state)
model_path = "hybrid_cnn_lstm_sentiment_model.keras"  # Update with your model's save path
model = load_model(model_path)

# Ensure class-specific performance
print("Class-Specific Performance on Test Set:")
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels
y_true = np.argmax(y_test, axis=1)  # Convert one-hot encoding to class labels

# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Negative', 'Neutral', 'Positive']))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# Check for overlap between training and test sets
print("\nChecking for Overlap Between Training and Test Sets:")
train_indices = np.arange(len(X_train))  # Indices used for training
test_indices = np.arange(len(X_train), len(X_train) + len(X_test))  # Indices used for testing

# Extract IDs for training and testing
train_ids = set(data.iloc[train_indices]['id'])
test_ids = set(data.iloc[test_indices]['id'])

overlap = train_ids.intersection(test_ids)
if overlap:
    print(f"Overlap Found: {len(overlap)} overlapping entries.")
else:
    print("No overlap found between training and test sets.")

# Cross-dataset testing (requires a new dataset)
new_file_path = "/content/us_election_comments_relabelled_final (1).csv"  # Update this with your new dataset's path
try:
    new_data = pd.read_csv(new_file_path)
    new_comments = new_data['comment'].values
    new_labels = new_data['label'].values + 1  # Shift labels from (-1, 0, 1) to (0, 1, 2)
    new_labels = to_categorical(new_labels, num_classes=3)

    # Tokenize and pad the new data
    new_sequences = tokenizer.texts_to_sequences(new_comments)
    new_padded = pad_sequences(new_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')

    # Evaluate the model on the new dataset
    print("\nEvaluating on New Dataset:")
    new_loss, new_accuracy = model.evaluate(new_padded, new_labels, verbose=1)
    print(f"New Dataset Loss: {new_loss}, New Dataset Accuracy: {new_accuracy}")

    # Detailed metrics
    new_pred_probs = model.predict(new_padded)
    new_pred = np.argmax(new_pred_probs, axis=1)
    new_true = np.argmax(new_labels, axis=1)

    print("Classification Report on New Dataset:")
    print(classification_report(new_true, new_pred, target_names=['Negative', 'Neutral', 'Positive']))
except FileNotFoundError:
    print("New dataset not found. Please provide a valid file path for cross-dataset testing.")


Class-Specific Performance on Test Set:
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Classification Report:
              precision    recall  f1-score   support

    Negative       0.95      1.00      0.97       129
     Neutral       1.00      0.99      0.99      1001
    Positive       0.98      0.99      0.99       494

    accuracy                           0.99      1624
   macro avg       0.98      0.99      0.98      1624
weighted avg       0.99      0.99      0.99      1624

Confusion Matrix:
[[129   0   0]
 [  6 987   8]
 [  1   3 490]]

Checking for Overlap Between Training and Test Sets:
Overlap Found: 1620 overlapping entries.

Evaluating on New Dataset:
272/272 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9697 - loss: 0.4331
New Dataset Loss: 0.39009591937065125, New Dataset Accuracy: 0.9726436734199524
272/272 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
Classification Report on New Dataset:
              precision    recall  f1-score   support

    Negative       0.93      1.00      0

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Conv1D, LSTM, GlobalMaxPooling1D, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = "/content/us_election_comments_relabelled_final (1).csv"  # Update with your file's path
data = pd.read_csv(file_path)

# Filter relevant comments and extract necessary columns
filtered_data = data[data['is_relevant'] == True][['id', 'comment', 'label']]

# Parameters
VOCAB_SIZE = 30000  # Increased vocabulary size
EMBEDDING_DIM = 300  # Larger embeddings for better feature representation
MAX_LENGTH = 200  # Allow longer sequences
BATCH_SIZE = 64  # Larger batch size for stability
EPOCHS = 30  # More epochs for better training

# Re-split the data to ensure no overlap
X = filtered_data['comment']
y = filtered_data['label'] + 1  # Shift labels from (-1, 0, 1) to (0, 1, 2)

# Ensure no overlap using stratified split
data_train, data_test, y_train, y_test = train_test_split(
    filtered_data, y, test_size=0.2, random_state=42, stratify=y)

# Extract training and testing comments
X_train = data_train['comment']
X_test = data_test['comment']

# Convert labels to categorical (for classification)
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

# Tokenize the text data
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the sequences
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

# Build the Hybrid CNN + LSTM model
model = Sequential([
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
    Conv1D(128, 5, activation='relu'),  # CNN layer to extract n-gram features
    Bidirectional(LSTM(128, activation='tanh', return_sequences=True)),  # LSTM layer for sequential context
    GlobalMaxPooling1D(),  # Global max pooling after LSTM
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Output layer with softmax for classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Suitable for multi-class classification
    metrics=['accuracy']  # Use accuracy as the primary metric
)

# Summary of the model
model.summary()

# Add early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the model in the recommended format
model.save("hybrid_cnn_lstm_sentiment_model_no_overlap.keras")

# Test predictions
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels
y_true = np.argmax(y_test, axis=1)  # Convert one-hot encoding to class labels

# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Negative', 'Neutral', 'Positive']))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# Optional: Cross-dataset testing
new_file_path = "/content/new_dataset.csv"  # Update this with your new dataset's path
try:
    new_data = pd.read_csv(new_file_path)
    new_comments = new_data['comment'].values
    new_labels = new_data['label'].values + 1  # Shift labels from (-1, 0, 1) to (0, 1, 2)
    new_labels = to_categorical(new_labels, num_classes=3)

    # Tokenize and pad the new data
    new_sequences = tokenizer.texts_to_sequences(new_comments)
    new_padded = pad_sequences(new_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')

    # Evaluate the model on the new dataset
    print("\nEvaluating on New Dataset:")
    new_loss, new_accuracy = model.evaluate(new_padded, new_labels, verbose=1)
    print(f"New Dataset Loss: {new_loss}, New Dataset Accuracy: {new_accuracy}")

    # Detailed metrics
    new_pred_probs = model.predict(new_padded)
    new_pred = np.argmax(new_pred_probs, axis=1)
    new_true = np.argmax(new_labels, axis=1)

    print("Classification Report on New Dataset:")
    print(classification_report(new_true, new_pred, target_names=['Negative', 'Neutral', 'Positive']))
except FileNotFoundError:
    print("New dataset not found. Please provide a valid file path for cross-dataset testing.")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_7 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_3               │ ?                           │     0 (unbuilt) │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.6308 - loss: 0.8573 - val_accuracy: 0.8812 - val_loss: 0.3235
Epoch 2/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.9216 - loss: 0.2470 - val_accuracy: 0.9631 - val_loss: 0.1295
Epoch 3/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.9788 - loss: 0.0679 - val_accuracy: 0.9828 - val_loss: 0.0641
Epoch 4/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.9979 - loss: 0.0109 - val_accuracy: 0.9883 - val_loss: 0.0794
Epoch 5/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.9989 - loss: 0.0055 - val_accuracy: 0.9828 - val_loss: 0.0751
Epoch 6/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.9999 - loss: 0.0019 - val_accuracy: 0.9889 - val_loss: 0.1069
Epoch 7/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.9994 - loss: 0.0013 - val_accuracy: 0.9877 - val_loss: 0.1003
Epoch 8/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 1.0000 - loss: 4.5641e-04 - val

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, f1_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define K-Fold Cross Validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store results
fold_accuracies = []
fold_f1_scores = []
fold_reports = []

# K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(filtered_data)):
    print(f"\nFold {fold + 1}/{kf.get_n_splits()}")

    # Split data
    train_data = filtered_data.iloc[train_index]
    val_data = filtered_data.iloc[val_index]

    X_train_fold = train_data['comment']
    y_train_fold = train_data['label'] + 1  # Shift labels from (-1, 0, 1) to (0, 1, 2)
    X_val_fold = val_data['comment']
    y_val_fold = val_data['label'] + 1

    # Tokenize and pad sequences
    X_train_seq = tokenizer.texts_to_sequences(X_train_fold)
    X_val_seq = tokenizer.texts_to_sequences(X_val_fold)
    X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
    X_val_padded = pad_sequences(X_val_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

    y_train_categorical = to_categorical(y_train_fold, num_classes=3)
    y_val_categorical = to_categorical(y_val_fold, num_classes=3)

    # Build the model for this fold
    model = Sequential([
        Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
        Conv1D(128, 5, activation='relu'),
        Bidirectional(LSTM(128, activation='tanh', return_sequences=True)),
        GlobalMaxPooling1D(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])

    # Compile the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    model.fit(
        X_train_padded, y_train_categorical,
        epochs=5,  # Use fewer epochs for cross-validation
        batch_size=BATCH_SIZE,
        verbose=0
    )

    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(X_val_padded, y_val_categorical, verbose=0)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Predict on validation data
    y_val_pred_probs = model.predict(X_val_padded)
    y_val_pred = np.argmax(y_val_pred_probs, axis=1)
    y_val_true = y_val_fold.values

    # Calculate F1-score
    fold_f1 = f1_score(y_val_true, y_val_pred, average='weighted')
    print(f"F1-Score: {fold_f1:.4f}")

    # Store results
    fold_accuracies.append(val_accuracy)
    fold_f1_scores.append(fold_f1)
    fold_reports.append(classification_report(y_val_true, y_val_pred, target_names=['Negative', 'Neutral', 'Positive'], output_dict=True))

# Print overall results
print("\nCross-Validation Results:")
print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f}, Std Dev: {np.std(fold_accuracies):.4f}")
print(f"Mean F1-Score: {np.mean(fold_f1_scores):.4f}, Std Dev: {np.std(fold_f1_scores):.4f}")



Fold 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Validation Loss: 0.0613, Validation Accuracy: 0.9883
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
F1-Score: 0.9883

Fold 2/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Validation Loss: 0.0647, Validation Accuracy: 0.9901
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
F1-Score: 0.9902

Fold 3/5


...

Validation Loss: 0.0697, Validation Accuracy: 0.9901
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
F1-Score: 0.9901

Fold 4/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Validation Loss: 0.0448, Validation Accuracy: 0.9920
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
F1-Score: 0.9920

Fold 5/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Validation Loss: 0.0656, Validation Accuracy: 0.9877
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
F1-Score: 0.9877

Cross-Validation Results:
Mean Accuracy: 0.9897, Std Dev: 0.0015
Mean F1-Score: 0.9897, Std Dev: 0.0015


gru


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = "/content/us_election_comments_relabelled_final (1).csv"  # Update with your file's path
data = pd.read_csv(file_path)

# Filter relevant comments and extract necessary columns
filtered_data = data[data['is_relevant'] == True][['comment', 'label']]
comments = filtered_data['comment'].values
labels = filtered_data['label'].values

# Convert labels to categorical (for classification)
labels = labels + 1  # Shift labels from (-1, 0, 1) to (0, 1, 2)
labels = to_categorical(labels, num_classes=3)

# Parameters
VOCAB_SIZE = 30000  # Increased vocabulary size
EMBEDDING_DIM = 300  # Larger embeddings for better feature representation
MAX_LENGTH = 200  # Allow longer sequences
BATCH_SIZE = 64  # Larger batch size for stability
EPOCHS = 30  # More epochs for better training

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(comments, labels, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the sequences
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

# Build the GRU model
model = Sequential([
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
    Bidirectional(GRU(128, activation='tanh', return_sequences=True)),
    Dropout(0.5),
    GRU(64, activation='tanh'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Output layer with softmax for classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Suitable for multi-class classification
    metrics=['accuracy']  # Use accuracy as the primary metric
)

# Summary of the model
model.summary()

# Add early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the model in the recommended format
model.save("gru_sentiment_model.keras")

# Test predictions
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels
y_true = np.argmax(y_test, axis=1)  # Convert one-hot encoding to class labels

# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Negative', 'Neutral', 'Positive']))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_14 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_13 (Bidirectional)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_31 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_32 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_33 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.5831 - loss: 0.9352 - val_accuracy: 0.6281 - val_loss: 0.8580
Epoch 2/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.6220 - loss: 0.8836 - val_accuracy: 0.6595 - val_loss: 0.8134
Epoch 3/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.6378 - loss: 0.8629 - val_accuracy: 0.6718 - val_loss: 0.8119
Epoch 4/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.6436 - loss: 0.8354 - val_accuracy: 0.6656 - val_loss: 0.8097
Epoch 5/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.6358 - loss: 0.8605 - val_accuracy: 0.6718 - val_loss: 0.7854
Epoch 6/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.6592 - loss: 0.7897 - val_accuracy: 0.8060 - val_loss: 0.5183
Epoch 7/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.8742 - loss: 0.3709 - val_accuracy: 0.9095 - val_loss: 0.1908
Epoch 8/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.9264 - loss: 0.1491 - val_accu

cross validation

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# Parameters
VOCAB_SIZE = 30000
EMBEDDING_DIM = 300
MAX_LENGTH = 200
BATCH_SIZE = 64
EPOCHS = 5  # Use fewer epochs for cross-validation

# Define K-Fold Cross Validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store results
fold_accuracies = []
fold_f1_scores = []
fold_reports = []

# K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(comments)):
    print(f"\nFold {fold + 1}/{kf.get_n_splits()}")

    # Split data
    X_train_fold = comments[train_index]
    y_train_fold = labels[train_index]
    X_val_fold = comments[val_index]
    y_val_fold = labels[val_index]

    # Tokenize and pad sequences
    tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train_fold)

    X_train_seq = tokenizer.texts_to_sequences(X_train_fold)
    X_val_seq = tokenizer.texts_to_sequences(X_val_fold)
    X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
    X_val_padded = pad_sequences(X_val_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

    # Build the GRU model
    model = Sequential([
        Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
        Bidirectional(GRU(128, activation='tanh', return_sequences=True)),
        Dropout(0.5),
        GRU(64, activation='tanh'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])

    # Compile the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    model.fit(
        X_train_padded, y_train_fold,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=0
    )

    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(X_val_padded, y_val_fold, verbose=0)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Predict on validation data
    y_val_pred_probs = model.predict(X_val_padded)
    y_val_pred = np.argmax(y_val_pred_probs, axis=1)
    y_val_true = np.argmax(y_val_fold, axis=1)

    # Calculate F1-score
    fold_f1 = f1_score(y_val_true, y_val_pred, average='weighted')
    print(f"F1-Score: {fold_f1:.4f}")

    # Store results
    fold_accuracies.append(val_accuracy)
    fold_f1_scores.append(fold_f1)
    fold_reports.append(classification_report(y_val_true, y_val_pred, target_names=['Negative', 'Neutral', 'Positive'], output_dict=True))

# Print overall results
print("\nCross-Validation Results:")
print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f}, Std Dev: {np.std(fold_accuracies):.4f}")
print(f"Mean F1-Score: {np.mean(fold_f1_scores):.4f}, Std Dev: {np.std(fold_f1_scores):.4f}")



Fold 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Validation Loss: 0.7967, Validation Accuracy: 0.6749
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
F1-Score: 0.5902

Fold 2/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Validation Loss: 0.8387, Validation Accuracy: 0.6359
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
F1-Score: 0.5258

Fold 3/5


...

Validation Loss: 0.8346, Validation Accuracy: 0.6525
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
F1-Score: 0.5512

Fold 4/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Validation Loss: 0.8001, Validation Accuracy: 0.6531
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
F1-Score: 0.5559

Fold 5/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Validation Loss: 0.8182, Validation Accuracy: 0.6476
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
F1-Score: 0.5567

Cross-Validation Results:
Mean Accuracy: 0.6528, Std Dev: 0.0127
Mean F1-Score: 0.5560, Std Dev: 0.0205


use

stratifiedkfold

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

# Define Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Compute class weights
y_classes = np.argmax(labels, axis=1)  # Convert one-hot to class labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_classes), y=y_classes)
class_weights = dict(enumerate(class_weights))

# Initialize tokenizer once
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(comments)

fold_accuracies, fold_f1_scores = [], []

for fold, (train_index, val_index) in enumerate(skf.split(comments, y_classes)):
    print(f"Fold {fold + 1}/5")

    # Split data
    X_train_fold, X_val_fold = comments[train_index], comments[val_index]
    y_train_fold, y_val_fold = labels[train_index], labels[val_index]

    # Tokenize and pad
    X_train_seq = tokenizer.texts_to_sequences(X_train_fold)
    X_val_seq = tokenizer.texts_to_sequences(X_val_fold)
    X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
    X_val_padded = pad_sequences(X_val_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

    # Build GRU model
    model = Sequential([
        Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
        Bidirectional(GRU(128, activation='tanh', return_sequences=True)),
        Dropout(0.5),
        GRU(64, activation='tanh'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train model
    model.fit(
        X_train_padded, y_train_fold,
        epochs=10,  # Slightly increase epochs
        batch_size=BATCH_SIZE,
        class_weight=class_weights,  # Apply class weights
        verbose=1
    )

    # Evaluate
    val_loss, val_accuracy = model.evaluate(X_val_padded, y_val_fold, verbose=1)
    y_val_pred_probs = model.predict(X_val_padded)
    y_val_pred = np.argmax(y_val_pred_probs, axis=1)
    y_val_true = np.argmax(y_val_fold, axis=1)

    # Metrics
    f1 = f1_score(y_val_true, y_val_pred, average='weighted')
    fold_accuracies.append(val_accuracy)
    fold_f1_scores.append(f1)
    print(f"Fold {fold + 1}: Accuracy={val_accuracy:.4f}, F1-Score={f1:.4f}")

# Summary
print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f}, Std Dev: {np.std(fold_accuracies):.4f}")
print(f"Mean F1-Score: {np.mean(fold_f1_scores):.4f}, Std Dev: {np.std(fold_f1_scores):.4f}")


Fold 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.4220 - loss: 1.0986
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.4185 - loss: 1.0857
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.4244 - loss: 1.0508
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.5128 - loss: 1.0021
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.4551 - loss: 1.0232
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.5340 - loss: 1.0053
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.4553 - loss: 1.0348
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.5326 - loss: 0.9900
Epoch 9/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.5773 - loss: 0.9969
Epoch 10/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.6137 - loss: 0.9937
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6524 - loss: 1.0512
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
Fold 1: Accura

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


102/102 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.3907 - loss: 1.1203
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.4518 - loss: 1.0931
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.4567 - loss: 1.0706
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.4378 - loss: 1.0395
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.5082 - loss: 1.0360
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.4970 - loss: 1.0145
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.5307 - loss: 1.0188
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.5424 - loss: 1.0161
Epoch 9/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.5998 - loss: 0.9935
Epoch 10/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.6103 - loss: 1.0128
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6668 - loss: 0.9692
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Fold 2: Accuracy=0.6482, F

...

102/102 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.4035 - loss: 1.1039
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.4209 - loss: 1.0976
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.4703 - loss: 1.0544
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.4827 - loss: 1.0175
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.5542 - loss: 1.0029
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.5728 - loss: 1.0049
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.6097 - loss: 1.0016
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.6112 - loss: 1.0354
Epoch 9/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.4663 - loss: 1.0339
Epoch 10/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.5997 - loss: 0.9722
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6479 - loss: 0.9095
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Fold 3: Accuracy=0.6488, F

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


102/102 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.3569 - loss: 1.1075
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.4579 - loss: 1.0669
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.3690 - loss: 1.1540
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.4098 - loss: 1.0328
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.4561 - loss: 1.0180
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.4550 - loss: 1.0267
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.5190 - loss: 1.0158
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.6011 - loss: 1.0225
Epoch 9/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.5770 - loss: 1.0115
Epoch 10/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.6345 - loss: 0.9904
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6412 - loss: 1.0430
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Fold 4: Accuracy=0.6482, F

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


102/102 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.4291 - loss: 1.1104
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.4851 - loss: 1.0608
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.4063 - loss: 1.0558
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.5329 - loss: 0.9976
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.4076 - loss: 1.0186
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.4942 - loss: 1.0301
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.4511 - loss: 1.0069
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.4443 - loss: 0.9851
Epoch 9/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.5269 - loss: 0.9913
Epoch 10/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.5549 - loss: 1.0262
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6682 - loss: 1.0358
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Fold 5: Accuracy=0.6550, F

change parameters

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight

# Parameters
VOCAB_SIZE = 30000
EMBEDDING_DIM = 300
MAX_LENGTH = 200
BATCH_SIZE = 64
EPOCHS = 15  # Increased epochs for better convergence
LEARNING_RATE = 0.0001  # Lower learning rate for stability

# Define Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Compute class weights
y_classes = np.argmax(labels, axis=1)  # Convert one-hot to class labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_classes), y=y_classes)
class_weights = dict(enumerate(class_weights))

# Initialize tokenizer once
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(comments)

fold_accuracies, fold_f1_scores = [], []

for fold, (train_index, val_index) in enumerate(skf.split(comments, y_classes)):
    print(f"Fold {fold + 1}/5")

    # Split data
    X_train_fold, X_val_fold = comments[train_index], comments[val_index]
    y_train_fold, y_val_fold = labels[train_index], labels[val_index]

    # Tokenize and pad
    X_train_seq = tokenizer.texts_to_sequences(X_train_fold)
    X_val_seq = tokenizer.texts_to_sequences(X_val_fold)
    X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
    X_val_padded = pad_sequences(X_val_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

    # Build the revised GRU model
    model = Sequential([
        Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
        Bidirectional(GRU(64, activation='tanh', return_sequences=True)),  # Reduced units for stability
        Dropout(0.6),  # Increased dropout
        GRU(32, activation='tanh'),
        Dropout(0.6),
        Dense(32, activation='relu'),
        Dropout(0.6),
        Dense(3, activation='softmax')
    ])

    # Compile the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    model.fit(
        X_train_padded, y_train_fold,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        class_weight=class_weights,  # Apply class weights
        verbose=1
    )

    # Evaluate
    val_loss, val_accuracy = model.evaluate(X_val_padded, y_val_fold, verbose=1)
    y_val_pred_probs = model.predict(X_val_padded)
    y_val_pred = np.argmax(y_val_pred_probs, axis=1)
    y_val_true = np.argmax(y_val_fold, axis=1)

    # Metrics
    f1 = f1_score(y_val_true, y_val_pred, average='weighted')
    fold_accuracies.append(val_accuracy)
    fold_f1_scores.append(f1)
    print(f"Fold {fold + 1}: Accuracy={val_accuracy:.4f}, F1-Score={f1:.4f}")

# Summary
print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f}, Std Dev: {np.std(fold_accuracies):.4f}")
print(f"Mean F1-Score: {np.mean(fold_f1_scores):.4f}, Std Dev: {np.std(fold_f1_scores):.4f}")


Fold 1/5
Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


102/102 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.4294 - loss: 1.0975
Epoch 2/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.3583 - loss: 1.1103
Epoch 3/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.4084 - loss: 1.0819
Epoch 4/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.3408 - loss: 1.0812
Epoch 5/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.4542 - loss: 1.0378
Epoch 6/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.4356 - loss: 1.0501
Epoch 7/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.4512 - loss: 1.0062
Epoch 8/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.4830 - loss: 1.0178
Epoch 9/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.5803 - loss: 0.9913
Epoch 10/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.5791 - loss: 0.9930
Epoch 11/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - accuracy: 0.5325 - loss: 1.0128
Epoch 12/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/ste

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


102/102 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.2946 - loss: 1.1103
Epoch 2/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.3797 - loss: 1.0912
Epoch 3/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.3159 - loss: 1.0910
Epoch 4/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.4520 - loss: 1.0630
Epoch 5/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.4005 - loss: 1.0710
Epoch 6/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.4769 - loss: 1.0464
Epoch 7/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.5329 - loss: 1.0228
Epoch 8/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.5412 - loss: 1.0191
Epoch 9/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.5672 - loss: 1.0309
Epoch 10/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.6173 - loss: 1.0126
Epoch 11/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.5964 - loss: 1.0158
Epoch 12/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/ste

...

102/102 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.3680 - loss: 1.1088
Epoch 2/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.4012 - loss: 1.0993
Epoch 3/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.4436 - loss: 1.0587
Epoch 4/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.3947 - loss: 1.0874
Epoch 5/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.4330 - loss: 1.0314
Epoch 6/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.4034 - loss: 1.0397
Epoch 7/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.4750 - loss: 1.0008
Epoch 8/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.4438 - loss: 1.0260
Epoch 9/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.5030 - loss: 1.0155
Epoch 10/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.5666 - loss: 1.0022
Epoch 11/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.5505 - loss: 1.0124
Epoch 12/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/ste

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


102/102 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.3303 - loss: 1.1276
Epoch 2/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.3461 - loss: 1.1047
Epoch 3/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.3790 - loss: 1.0769
Epoch 4/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.3373 - loss: 1.0745
Epoch 5/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 41ms/step - accuracy: 0.3938 - loss: 1.0680
Epoch 6/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.3586 - loss: 1.0445
Epoch 7/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.4191 - loss: 1.0288
Epoch 8/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.4831 - loss: 1.0579
Epoch 9/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.5272 - loss: 1.0268
Epoch 10/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.5275 - loss: 1.0218
Epoch 11/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.6173 - loss: 0.9851
Epoch 12/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/ste

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


102/102 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.4158 - loss: 1.0780
Epoch 2/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.3596 - loss: 1.1065
Epoch 3/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.3558 - loss: 1.0922
Epoch 4/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.4442 - loss: 1.0541
Epoch 5/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.4080 - loss: 1.0437
Epoch 6/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.4559 - loss: 1.0287
Epoch 7/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - accuracy: 0.4836 - loss: 0.9953
Epoch 8/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.5444 - loss: 1.0233
Epoch 9/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.5367 - loss: 1.0283
Epoch 10/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.5770 - loss: 1.0083
Epoch 11/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.5901 - loss: 1.0049
Epoch 12/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/ste